In [59]:
#import stuff 
import wordle_words as wordle
from collections import Counter
from tqdm import tqdm

In [90]:
# new way of inputting information 

# Here to input information, put in your five letter guess, 
# followed by the response in the following way: 
#  put ('guess','response'), on the list, where each character of 'response' 
#  indicates the color that the corresponding character of the guess turned: 
#    *  if the Tile turned green,  put 'g'
#    *  if the Tile turned yellow, put 'y'
#    *  if the Tile turned gray,   put 'x' 
# so for example if you guessed 'tears', and the correct word was 'rebut', 
#       the response string would be 'ygxyx'

# You should always start with the guess 'tears', rather than re-running this program 

Guesses = [
    ('tears','ygxyx'),  
    ('micht','xxxxg'),
    ('elbow','yxgxx')
]

In [91]:
known_letters = {}
yellow_letters = []
wrong_letters = ''
for guess,resp in Guesses: 
    for i in range(5): 
        if resp[i] == 'g': 
            known_letters[i] = guess[i]
        elif resp[i] == 'y': 
            yellow_letters.append((i,guess[i]))
        else: 
            wrong_letters += guess[i]
            
print(known_letters)
print(yellow_letters)
print(wrong_letters)

{1: 'e', 4: 't', 2: 'b'}
[(0, 't'), (3, 'r'), (0, 'e')]
asmichlow


In [92]:
possible_words = wordle.words

# start by narrowing based off of known letters 
for ind, val in known_letters.items():
    if val == '' : 
        continue 
    print(f"narrowing based off of '{val}' in spot {ind}")
    possible_words = [word for word in  possible_words if word[ind] == val]
    print(f"\t list size is {len(list(possible_words))}")

# then look at all yellow letters 
for ind, val in yellow_letters:
    if val == '' : 
        continue 
    print(f"narrowing based off of '{val}' in spot {ind}")
    possible_words = [word for word in  possible_words if val in word and word[ind] != val]
    print(f"\t list size is {len(list(possible_words))}")
# finally look at all letters not in the words 

print("narrowing words based off of incorrect letters")
for let in wrong_letters: 
#     print(f"narrowing based off of incorrect letter '{let}'")
    possible_words = [word for word in  possible_words if let not in word ]
#     print(f'list is {possible_words}')
print(f"\t list size is {len(list(possible_words))}")


print(f"all possible words are: {list(possible_words)}")

narrowing based off of 'e' in spot 1
	 list size is 1628
narrowing based off of 't' in spot 4
	 list size is 86
narrowing based off of 'b' in spot 2
	 list size is 4
narrowing based off of 't' in spot 0
	 list size is 4
narrowing based off of 'r' in spot 3
	 list size is 2
narrowing based off of 'e' in spot 0
	 list size is 2
narrowing words based off of incorrect letters
	 list size is 1
all possible words are: ['rebut']


In [89]:
# hard mode based geussing
hard_mode = False # currently hard mode is not garunteed to be 'optimal'. 
                  # But when only a few options are avaliable, it often performs better.

possible_guesses = possible_words  if hard_mode else wordle.nltk_words
best_num_uniques = 0 
best_guesses = []
for guess in tqdm(possible_guesses):
#     print(f'guess is {guess}')
    resps = []
    for ans in possible_words: 
        resp = ['g' if ans[i] == c else 'y' if c in ans else 'x' for i,c in enumerate(guess)]
#         print(f"word is {ans}, ans is '{''.join(resp)}'")

        resps.append(''.join(resp))
    num_uniques = len(Counter(resps))
    if num_uniques > best_num_uniques: 
        best_guesses = []
        best_num_uniques = num_uniques
    if num_uniques >= best_num_uniques: 
        entr = {
            "guess":        guess,
            "num_uniques":  num_uniques, 
            "resp_distro":  Counter(resps)
        }
        best_guesses.append(entr)
    
for guess in best_guesses:
    printafterline=False
    if guess['guess'] in possible_words and not hard_mode:
        printafterline = True 
        print("Reccomended word: ")
    print(guess)
    if printafterline : 
        print(" ")
        
    

100%|██████████| 8689/8689 [00:00<00:00, 43590.54it/s]

{'guess': 'balow', 'num_uniques': 6, 'resp_distro': Counter({'gxxxx': 1, 'yxxxx': 1, 'xxgxx': 1, 'xxxgx': 1, 'xxxxx': 1, 'xxxxy': 1})}
{'guess': 'below', 'num_uniques': 6, 'resp_distro': Counter({'ggxxx': 1, 'ygxxx': 1, 'xggxx': 1, 'xgxgx': 1, 'xgxxx': 1, 'xgxxy': 1})}
{'guess': 'bevel', 'num_uniques': 6, 'resp_distro': Counter({'ggxgx': 1, 'ygxyx': 1, 'xgxgy': 1, 'xgxyx': 1, 'xgggx': 1, 'xgxgx': 1})}
{'guess': 'blown', 'num_uniques': 6, 'resp_distro': Counter({'gxxxx': 1, 'yxxxx': 1, 'xyxxx': 1, 'xxyxx': 1, 'xxxxx': 1, 'xxxyx': 1})}
{'guess': 'blowy', 'num_uniques': 6, 'resp_distro': Counter({'gxxxx': 1, 'yxxxx': 1, 'xyxxx': 1, 'xxyxx': 1, 'xxxxx': 1, 'xxxyx': 1})}
{'guess': 'bowel', 'num_uniques': 6, 'resp_distro': Counter({'gxxgx': 1, 'yxxyx': 1, 'xxxgy': 1, 'xyxyx': 1, 'xxxgx': 1, 'xxggx': 1})}
{'guess': 'bowla', 'num_uniques': 6, 'resp_distro': Counter({'gxxxx': 1, 'yxxxx': 1, 'xxxyx': 1, 'xyxxx': 1, 'xxxxx': 1, 'xxgxx': 1})}
{'guess': 'bowls', 'num_uniques': 6, 'resp_distro': Cou

In [29]:
print(len(wordle.words))
print(len(wordle.nltk_words))
print(len(wordle.sc_words))

12972
8689
2315
